In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from google.cloud import bigquery
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

In [2]:
query="""
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 10000
"""
df = bigquery.Client().query(query).to_dataframe()
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,7.936641,False,30,1,38.0
1,8.624484,True,34,1,39.0
2,6.937947,True,23,1,39.0
3,6.935743,True,18,1,39.0
4,8.318041,False,23,1,40.0


In [3]:
df.describe()

,weight_pounds,mother_age,plurality,gestation_weeks
count,9995.000000,10000.0000,10000.000000,9885.000000
mean,7.263349,27.1810,1.035300,38.705311
std,1.315565,6.1537,0.196616,2.567750
min,0.507063,12.0000,1.000000,18.000000
25%,6.624891,22.0000,1.000000,38.000000
50%,7.374463,27.0000,1.000000,39.000000
75%,8.062305,32.0000,1.000000,40.000000
max,12.625874,48.0000,4.000000,47.000000


In [4]:
df['is_male'].value_counts()

True     5198
False    4802
Name: is_male, dtype: int64

In [5]:
df = df.dropna()
df = shuffle(df, random_state=2)

In [6]:
labels = df['weight_pounds']
data = df.drop(columns=['weight_pounds'])
data['is_male'] = data['is_male'].astype(int)

In [7]:
x,y = data,labels
x_train,x_test,y_train,y_test = train_test_split(x,y)

In [8]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(len(x_train.iloc[0]),)),
    Dense(32, activation='relu'),
    Dense(1)]
)

In [9]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mae', 'mse'])

In [10]:
model.fit(x_train, y_train, epochs=10, validation_split=0.1)

Epoch 1/10
209/209 [==============================] - 2s 9ms/step - loss: 1.6824 - mae: 0.9623 - mse: 1.6824 - val_loss: 1.1770 - val_mae: 0.8422 - val_mse: 1.1770
Epoch 2/10
209/209 [==============================] - 1s 6ms/step - loss: 1.3390 - mae: 0.8925 - mse: 1.3390 - val_loss: 1.1442 - val_mae: 0.8314 - val_mse: 1.1442
Epoch 3/10
209/209 [==============================] - 1s 5ms/step - loss: 1.2990 - mae: 0.8840 - mse: 1.2990 - val_loss: 1.7296 - val_mae: 1.0589 - val_mse: 1.7296
Epoch 4/10
209/209 [==============================] - 1s 6ms/step - loss: 1.2912 - mae: 0.8780 - mse: 1.2912 - val_loss: 1.1229 - val_mae: 0.8233 - val_mse: 1.1229
Epoch 5/10
209/209 [==============================] - 1s 6ms/step - loss: 1.2749 - mae: 0.8719 - mse: 1.2749 - val_loss: 1.1579 - val_mae: 0.8408 - val_mse: 1.1579
Epoch 6/10
209/209 [==============================] - 1s 5ms/step - loss: 1.2467 - mae: 0.8612 - mse: 1.2467 - val_loss: 1.3903 - val_mae: 0.9345 - val_mse: 1.3903
Epoch 7/10
209/2

In [11]:
num_examples = 10
predictions = model.predict(x_test[:num_examples])

In [12]:
for i in range(num_examples):
    print('Predicted val: ', predictions[i][0])
    print('Actual val: ',y_test.iloc[i])
    print()

Predicted val:  7.0782566
Actual val:  5.8135898489399995

Predicted val:  7.2651606
Actual val:  5.8753192823

Predicted val:  6.94027
Actual val:  4.68702769012

Predicted val:  7.8132625
Actual val:  8.50102482272

Predicted val:  7.7818494
Actual val:  8.437090766739999

Predicted val:  7.5121326
Actual val:  7.06361087448

Predicted val:  7.9895473
Actual val:  7.25100379718

Predicted val:  8.127785
Actual val:  8.000575487979999

Predicted val:  7.7125745
Actual val:  8.062304921339999

Predicted val:  7.5462675
Actual val:  8.062304921339999

